# Analysis of Divvy Usage and Chicago Weather

Joint analysis of Divvy bikesharing data and Chicago weather from April 2020 to May 2023. [View this notebook on NBViewer](https://nbviewer.org/github/pollyren/divvy/blob/main/analysis/weather_analysis.ipynb) to see the proper map renderings.

### Preliminaries

In [1]:
import pandas as pd
import os
from datetime import datetime
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import folium

In [58]:
dtypes = {
    'temp': float,
    'feelslike': float,
    'dew': float,
    'humidity': float,
    'precip': float,
    'snowdepth': float,
    'windspeed': float,
    'winddir': float,
    'sealevelpressure': float,
    'cloudcover': float,
    'visibility': float,
    'solarradiation': float,
    'solarenergy': float,
    'uvindex': float,
    'severerisk': str,
    'conditions': str,
    'icon': str,
    'year': int,
    'month': int,
    'day': int,
    'hour': int,
    'member_classic_counts': float,
    'member_electric_counts': float,
    'casual_classic_counts': float,
    'casual_electric_counts': float,
    'member_classic_avg_time': float,
    'member_electric_avg_time': float,
    'casual_classic_avg_time': float,
    'casual_electric_avg_time': float,
    'member_classic_avg_dist': float,
    'member_electric_avg_dist': float,
    'casual_classic_avg_dist': float,
    'casual_electric_avg_dist': float,
    'pop_station': str
}

In [61]:
data_path = os.getcwd() + '/../data/'
data = pd.read_csv(data_path+'bike_weather_merged.csv', dtype=dtypes, index_col=0)

In [62]:
data.head()

,temp,feelslike,dew,humidity,precip,snowdepth,windspeed,winddir,sealevelpressure,cloudcover,...,casual_electric_counts,member_classic_avg_time,member_electric_avg_time,casual_classic_avg_time,casual_electric_avg_time,member_classic_avg_dist,member_electric_avg_dist,casual_classic_avg_dist,casual_electric_avg_dist,pop_station
0,38.6,32.9,34.4,84.67,0.0,0.0,8.0,357.0,1018.1,95.5,...,NaN,6.488889,NaN,22.100000,NaN,0.797176,NaN,1.992716,NaN,Kingsbury St & Erie St
1,38.6,32.2,34.4,84.67,0.0,0.0,9.5,360.0,1017.9,99.3,...,NaN,7.141667,NaN,NaN,NaN,0.885256,NaN,NaN,NaN,May St & Taylor St
2,38.6,32.7,34.4,84.67,0.0,0.0,8.4,358.0,1017.6,89.4,...,NaN,13.116667,NaN,31.466667,NaN,0.000000,NaN,2.370126,NaN,Michigan Ave & Madison St
3,38.6,33.0,34.1,83.77,0.0,0.0,7.8,327.0,1017.6,75.0,...,NaN,12.933333,NaN,21.916667,NaN,1.327151,NaN,0.620873,NaN,Emerald Ave & 28th St
4,38.6,33.2,34.1,83.77,0.0,0.0,7.4,358.0,1017.6,70.7,...,NaN,8.016667,NaN,NaN,NaN,1.014499,NaN,NaN,NaN,Rush St & Cedar St
